In [ ]:
!pip install wandb==0.13.9
!pip install pandas-profiling==3.6.2
!pip install seaborn==0.11.1
!pip install pandas=1.2.3
!pip install pyarrow=2.0

1. Fetch the artifact we just created (sample.csv) from W&B and read it with pandas:

In [ ]:
import wandb
import pandas as pd
# Note that we use save_code=True in the call to wandb.init so the notebook is uploaded and versioned by W&B
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

2. Using pandas-profiling, create a profile:

In [ ]:
import pandas_profiling

profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

3. What do you notice in the data? Look around and see what you can find.

> For example, there are missing values in a few columns and the column `last_review` is a date but it is in string format. Look also at the `price` column, and note the outliers. There are some zeros and some very high prices. After talking to your stakeholders, you decide to consider from a minimum of $10 to a maximum of $350 per night.

4. Fix some of the little problems we have found in the data with the following code:

In [ ]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

Note how we did not impute missing values. We will do that in the inference pipeline, so we will be able to handle missing values also in production.

5. Create a new profile or check with df.info() that all obvious problems have been solved

In [ ]:
df.info()

6. Terminate the run by running `run.finish()`

In [ ]:
run.finish()

7. Save the notebook.